![image](./../static/langgraph_adaptive_rag.png)

Q. what is adaptive rag?<br>
A. adaptive rag a fancy word of simply using a question router to route our question to different rack flows. <br> 
So in we're going to be using two flows. <br> 
So the first one is going to be taking the route to search on the internet and then to continue downstream on the same logic we have before. <br> 
And the second round is going to be to use the retrieval augmentation from our vector store. <br>
So we're first going to take the user's question, and we're going to decide whether the information is stored in the vector store to answer that question. And if it's not, we're simply going to take the route to web search and answer from there. <br>
you use `question_router` chain in `./chains/router.py` script to decide which route is more appropriate for this question.

### Q. explain Literal in question_router chain?
The Literal type from the typing module in Python specifies that a variable must be equal to one of a specific set of values. <br>
Literal is a type hint from Python's typing module.<br>
During RunTime, it may give you an error, like the exmple below. but with static type checker, it raises an error. <br>

--> ex that violates Lieral type hinting:
```python
var1 :Literal["ali", "reza"] = "ahmad"

```

In question_router:<br>

```python
datasource: Literal["vectorstore", "websearch"]
```

This means the datasource field must be either `"vectorstore" or "websearch"—nothing` else is allowed. It enforces strict value checking at both runtime (via Pydantic) and via type hints for tools like linters or IDEs.

## question_router chain:
```python
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI


class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["vectorstore", "websearch"] = Field(
        ...,
        description="Given a user question choose to route it to web search or a vectorstore.",
    )


llm = ChatOpenAI(temperature=0)
structured_llm_router = llm.with_structured_output(RouteQuery)

system = """You are an expert at routing a user question to a vectorstore or web search.
The vectorstore contains documents related to agents, prompt engineering, and adversarial attacks.
Use the vectorstore for questions on these topics. For all else, use web-search."""
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

question_router = route_prompt | structured_llm_router
```

In [1]:
from typing import Literal

var1 :Literal["ali", "reza"] = "ali"

In [3]:
type(var1)

str

In [2]:
var1 :Literal["ali", "reza"] = "ahmad"